# AI Assistant

In this notebook you'll make an AI assistant to help customers make the best decision about getting a new shoe from Cloudrunners Shoes. You will also take a short dive into practical token limitations for the model you are working with, and where you can focus your learning in the case you need to build more powerful AI agent systems.

### Amazon Bedrock

In [2]:
import boto3, json
br_client = boto3.client('bedrock-runtime')

def generate(prompt, temperature = 0):
    body = json.dumps({
        "prompt": prompt,
        "temperature": temperature,
        "top_p": 0.9,
        "max_gen_len":512
    })
    response = br_client.invoke_model(body=body, modelId='meta.llama2-13b-chat-v1')
    response = json.loads(response.get('body').read())
    response = response.get('generation')
    return response

### Edit System Message

In [3]:
def prompt_with_system_message(prompt, system_message):
    prompt = f"""
    <s>[INST] <<SYS>>{system_message}<</SYS>>

    User: {prompt}
    Agent:[/INST]
    """
    return prompt

### Include One-to-many Shot Learning

In [4]:
def prompt_with_examples(prompt, system_message, examples=[]):
    
    # Start with the initial part of the prompt with system message
    full_prompt = f"<s>[INST] <<SYS>>{system_message}<</SYS>>\n"

    # Add each example to the prompt
    for user_msg, agent_response in examples:
        full_prompt += f"{user_msg} [/INST] {agent_response} </s><s>[INST]"

    # Add the main prompt and close the template
    full_prompt += f"{prompt} [/INST]"

    return full_prompt

### LlamaChatbot Class

In [5]:
class LlamaChatbot:
    def __init__(self, system_message):
        self.system_message = system_message
        self.conversation_history = []  # list of tuples (user_msg, agent_response)

    def chat(self, user_msg):
        # Generate the prompt using the conversation history and the new user message
        prompt = prompt_with_examples(user_msg, self.system_message, self.conversation_history)
        
        # Get the model's response
        agent_response = generate(prompt)

        # Store this interaction in the conversation history
        self.conversation_history.append((user_msg, agent_response))

        return agent_response

    def reset(self):
        # Clear conversation history
        self.conversation_history = []

## Data

### Cloudrunners Shoes Details

In [6]:
shoes = [
    {
        "model": "Sky Glider",
        "type": "Running", 
        "features": {
            "upper": "Mesh",
            "sole": "EVA foam",
            "lacing": "Quick-tie",
            "drop": "8mm",
            "color": "Blue with yellow accents"
        },
        "usps": ["Lightweight", "Responsive cushioning", "Seamless upper"],
        "price": 119.95,
        "internal_id": "SG123",
        "weight": "220g",
        "manufacturer_location": "Vietnam"
    },
    {   
        "model": "Trail Trekker",
        "type": "Hiking",
        "features": {
            "upper": "Synthetic leather",
            "sole": "Rubber lug", 
            "lacing": "Traditional",
            "drop": "12mm",
            "color": "Khaki green"
        },
        "usps": ["Rugged construction", "Super grippy sole", "Waterproof"], 
        "price": 129.99,
        "internal_id": "TT321",
        "weight": "340g",
        "manufacturer_location": "China"
    },
    {
        "model": "Marathon Master",
        "type": "Racing",
        "features": {
            "upper": "Mesh",
            "sole": "Carbon fiber plate",
            "lacing": "Speed laces",
            "drop": "6mm",
            "color": "Neon yellow and black"
        },
        "usps": ["Maximizes energy return", "Lightning fast", "Seamless comfort"],
        "price": 179.50, 
        "internal_id": "MM111",
        "weight": "180g",
        "manufacturer_location": "USA"
    }
]

## Shoes AI Assistant

In [7]:
system_message = """
You are a friendly chatbot knowledgeable about shoes. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [8]:
print(chatbot.chat("Can you tell me about the latest models?"))

 Oh my soles! 😍 There are so many amazing shoe models out there right now! 😍 As a friendly chatbot, I'd love to help you find the perfect pair. 👠👞

If you're looking for the latest trends, you should definitely check out the newest sneaker drops from brands like Nike, Adidas, and Converse. 👟👞 They have some amazing styles and colors that are perfect for both men and women.

For women, I recommend checking out the Nike Air Force 1 Low in the "Triple Black" colorway. 🔥 It's a classic style that's both timeless and on-trend. Plus, it's super comfortable and versatile, so you can wear it with just about anything. 👖👗

For men, the Adidas Yeezy Boost 350 V2 in "Cream White" is a must-have. 😍 It's a stylish and comfortable sneaker that's perfect for everyday wear. Plus, the cream white colorway is super on-trend right now. 💥

If you're looking for something a bit more formal, you should check out the new line of dress shoes from Allen Edmonds. 👞 They have some amazing styles that are perfect 

In [9]:
chatbot.reset()

## Cloudrunners Shoes AI Assistant

In [10]:
system_message = f"""
You are a friendly chatbot knowledgeable about these bicycles from Cloudrunners Shoes {shoes}. \
When asked about specific shoe models or features, you try to provide accurate and helpful answers. \
Your goal is to assist and inform potential customers to the best of your ability.
"""

chatbot = LlamaChatbot(system_message)

In [11]:
print(chatbot.chat("Can you tell me about the latest models?"))

 Absolutely! Cloudrunners Shoes has recently released some exciting new models that are sure to catch your eye. Let me tell you about them! 😊

First up, we have the all-new "Sky Glider" model, which is a running shoe designed for lightweight, responsive performance. It features a mesh upper with EVA foam cushioning and quick-tie lacing for a snug, comfortable fit. The drop is 8mm, and the shoe weighs just 220g, making it perfect for runners who want to fly through their routes. The best part? It's priced at an affordable $119.95! 💰

Next, we have the "Trail Trekker," a hiking shoe designed for rugged terrain. It features a synthetic leather upper with rubber lug soles for maximum grip and traction. The shoe also has traditional lacing and a 12mm drop, making it perfect for long hikes on uneven terrain. And at just $129.99, it's a steal! 🏔️

Last but not least, we have the "Marathon Master," a racing shoe designed for speed and performance. It features a mesh upper with a carbon fiber p

In [12]:
print(chatbot.chat("How much do each of the models cost?"))

 Great question! Here are the prices for each of the models:

* Sky Glider: $119.95
* Trail Trekker: $129.99
* Marathon Master: $179.50

I hope that helps! Do you have any other questions about these models or any other questions in general? 😊


In [13]:
print(chatbot.chat("I'm torn between the Sky Glider and the Marathon Master, how should I decide which is best or me?"))

 That's a tough decision! Both the Sky Glider and the Marathon Master are great options, but they cater to different needs and preferences. Here are some factors to consider that might help you make a decision:

Sky Glider:

* Lightweight and responsive: If you're looking for a shoe that's easy to run in and provides a snappy, responsive feel, the Sky Glider might be the better choice.
* Ideal for shorter distances: The Sky Glider is designed for shorter distances, so if you're a sprinter or a casual runner who only runs a few miles at a time, this shoe might be perfect for you.
* Affordable price: At $119.95, the Sky Glider is a great value for the price.

Marathon Master:

* Maximum energy return: If you're a long-distance runner who needs a shoe that can help you maintain your energy and speed over the course of a long race, the Marathon Master might be the better choice. The carbon fiber plate provides maximum energy return and helps you maintain your speed.
* Supportive and stable

In [14]:
print(chatbot.chat("I'm torn between the Marathon Master and the Trail Trekker, how should I decide which is best or me?"))

 That's a tough decision! Both the Marathon Master and the Trail Trekker are great options, but they cater to different needs and preferences. Here are some factors to consider that might help you make a decision:

Marathon Master:

* Maximum energy return: If you're a long-distance runner who needs a shoe that can help you maintain your energy and speed over the course of a long race, the Marathon Master might be the better choice. The carbon fiber plate provides maximum energy return and helps you maintain your speed.
* Supportive and stable: The Marathon Master has a more supportive and stable design, which makes it a great choice for runners who need a little more support or stability during their runs.
* Premium materials: The Marathon Master is made with high-quality materials, including a mesh upper and a carbon fiber plate, which makes it a more premium option.

Trail Trekker:

* Rugged construction: If you're a trail runner who needs a shoe that can handle rough terrain and un

In [16]:
chatbot.reset()